In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from get_args import get_parser
import torch
from transformers import BertTokenizerFast
from dataset import *
from train import *
from model import *
import random
import numpy as np
import torch.nn as nn

In [ ]:
class Args:
    model_name = 'hfl/chinese-roberta-wwm-ext'
    max_length = 128
    N = 15
args = Args()

In [ ]:
model = LRBModel.from_pretrained(args.model_name, num_labels=2,max_length = args.max_length)
model.load_state_dict(torch.load('./ckpts/lrb_epoch_1.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
sent = '机器学习是一种很牛逼的方法。'
sents = [list(sent),]
tk = BertTokenizerFast.from_pretrained(args.model_name)
inp = tk(
    sents,
    padding='max_length',
    truncation=True,
    max_length=args.max_length,
    is_split_into_words=True,
    return_tensors='pt'
)
# inp
b_input_ids = inp['input_ids'].to(device)
b_input_mask = inp['attention_mask'].to(device)

outputs = model(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                labels=None)
bs,seqlen,hs = outputs[0].shape
logits = outputs[0].detach().cpu().numpy().argmax(axis=-1).reshape(bs,2,seqlen//2)
extracted_terms = []
for logit,sent in zip(logits,sents):
    sent_len = min(len(sent),args.max_length-2)
    sent_ngram_idxes = get_ngram_idxes(args.N,sent_len)
    for i,j in sent_ngram_idxes:
        if logit[0,i]==1 and logit[1,j+1]==1:
            extracted_terms.append(''.join(sent[i:j+1]))
extracted_terms

In [ ]:
tmp = [1,2]
newt = tmp+ [0]*-1
newt

In [ ]:
get_ngram_idxes(3,5)

In [1]:
recall = 2.3
precision = 95.3
f1 = 2*recall*precision/(recall+precision)
f1

4.491598360655737